In [ ]:
from flask import Flask, request, jsonify, send_file
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
import tensorflow as tf
import requests
from io import BytesIO

app = Flask(__name__)


# Load the pre-trained model
# food_quality_model.h5 이 다른 디렉토리에 있는 경우
# model_path = '/absolute/path/to/food_quality_model.h5'
#loaded_model = tf.keras.models.load_model(model_path)

# food_quality_model.h5 이 똑같은 디렉토리에 있는 경우
loaded_model = tf.keras.models.load_model('food_quality_model.h5')
loaded_model.summary()

# 이미지 평가 및 재구성 함수
def evaluate_and_reconstruct_image(image_url):
    # 이미지 가져오기
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

# 이미지 크기를 (224, 224)로 조정하고 채널 수를 3으로 설정
    image = image.resize((224, 224))
    image_array = np.array(image)

    # 이미지 전처리 및 예측
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array /= 255.0
    prediction = model.predict(image_array)

    # 카테고리별 피드백 및 점수
    categories = ['구도', '명도', '채도', '선명도', '색온도']
    feedback = []
    scores = []

    # 각 카테고리에 대한 피드백 및 점수 생성
    for i in range(len(categories)):
        score = int((prediction[0, i]) * 10)
        scores.append(score)

        if i == 0:
            max_index = np.argmax(prediction[0, 1:5])
            shift_directions = ['위쪽', '아래쪽', '왼쪽', '오른쪽']
            feedback.append(f"{categories[i]}: 사진을 {shift_directions[max_index]}으로 움직여 보세요!")
        else:
            if score < 0:
                feedback.append(f"{categories[i]}를 높여주세요!")
            else:
                feedback.append(f"{categories[i]}를 낮춰주세요!")

    return feedback, scores

# 이미지 평가 API 엔드포인트
@app.route('/evaluate_image', methods=['POST'])
def evaluate_and_reconstruct_image_endpoint():
    data = request.get_json()
    image_url = data.get('image_url')

    feedback, reconstructed_image_filename = evaluate_and_reconstruct_image(image_url)

    # 추가: reconstructed_image_filename를 문자열로 변환
    reconstructed_image_filename = str(reconstructed_image_filename)

    # 재구성된 이미지 파일의 경로 반환
    reconstructed_image_path = os.path.join('static', reconstructed_image_filename)

    return jsonify({'feedback': feedback, 'reconstructed_image_path': reconstructed_image_filename})

# 재구성된 이미지 접근을 위한 엔드포인트
@app.route('/static/<filename>')
def reconstructed_image(filename):
    return send_file(os.path.join('static', filename), mimetype='image/jpeg')

# Flask 애플리케이션 실행
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0',port=5001)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found 1906 images belonging to 5 classes.


In [ ]:
# CNN 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 4)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_generator, epochs=1)

# 모델 저장
#model.save('food_quality_model.h5')

InvalidArgumentError: ignored